## Face Extraction
- working with only one file at a time moment

In [ ]:
import numpy as np
import argparse
from cv2 import cv2
import os
import glob

In [ ]:
print("[INFO] loading face detector...")
protoPath = "../res/face_detector/deploy.prototxt" 
modelPath = "../res/face_detector/res10_300x300_ssd_iter_140000.caffemodel" 
net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

In [ ]:
with os.scandir("../data/fake/") as it:
    for entry in it:
        if entry.name != ".DS_Store" and entry.is_file():
            vs = cv2.VideoCapture(entry.path)
            vs.set(cv2.CAP_PROP_FPS, 60)
            print(entry.name)

            fake_file_path = "../dataset/fake/*.png*"

            max_file_fullpath = (max(glob.glob(fake_file_path), key=os.path.getmtime))
            max_file = os.path.basename(max_file_fullpath)
            max_filename = int(max_file.split(".")[0])
            read = 0
            saved = max_filename+1 #manual
            (grabbed, frame) = vs.read()
            read += 1

            (h, w) = frame.shape[:2]
            blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
                (300, 300), (104.0, 177.0, 123.0))
            net.setInput(blob)
            detections = net.forward()

            if len(detections) > 0:
                i = np.argmax(detections[0, 0, :, 2])
                confidence = detections[0, 0, i, 2]

                if confidence > 0.5:
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")
                    face = frame[startY:endY, startX:endX]

                    p = os.path.sep.join(['../dataset/fake',
                        "{}.png".format(saved)])
                    cv2.imwrite(p, face)
                    saved += 1
                    print("[INFO] saved {} to disk".format(p))

In [ ]:
vs.release()
cv2.destroyAllWindows()